In [ ]:
#downoad data
# ! pip install pandas
# ! pip install sod
# ! pip install 
# ! pip install seaborn

# Develop predictive model for disability rates by demographic for better interventions and support services

In this project, we aim to uncover patterns and disparities that might otherwise go unnoticed by developing a predictive data science model on metadata published by CDC from 2010. The disability rates and numbers are age weighted, so we can rule out the bias from age in the dataset.The data-driven approaches empower us to make informed decisions that enhance the equality of life for individuals with disabilities.
To achieve the above objectives, we develop a data pipeline starting with data cleaning and analysis, transform features to meaningful date types (such as, categorical to hot-encoded features and transform numeric data into float type), evaluate and explore the hidden trends among features and target variable to provide new insights from the data, visualization of the trends help reveal the trends. Explored the correlation between selected features with target variable (data_value in percentage), so that we can see what features have higher correlations with target variable. Next, we created a predictive model based on selected features and evaluated the model accuracy on test dataset. Finally, we stored the selected features and target data as metadata with description in a readme.txt file, and shared our whole data analysis in github repository. 


In [ ]:
import pandas as pd
import plotly.express as px
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:

from sodapy import Socrata

# Unauthenticated client only works with public data sets. Note 'None'
# in place of application token, and no username or password:
client = Socrata("data.cdc.gov", None)

# Example authenticated client (needed for non-public datasets):
# client = Socrata(data.cdc.gov,
#                  MyAppToken,
#                  username="user@example.com",
#                  password="AFakePassword")

# First 2000 results, returned as JSON from API / converted to Python list of
# dictionaries by sodapy.
results = client.get("qjg3-6acf", limit=2000)

# Convert to pandas DataFrame
results_df = pd.DataFrame.from_records(results)

In [ ]:
results_df = pd.read_csv('cdc_preventive_disability_2010.csv', header=0)
results_df.head(2)

# Data type and missing data 

In [ ]:
results_df.shape

In [ ]:
results_df.describe()

In [ ]:
results_df.info()

In [ ]:
results_df.indicator.unique()

In [ ]:
results_df.response.unique()

In [ ]:
# convert data_value into float type
results_df.data_value = pd.to_numeric(results_df.data_value)

# Data EDA and visualization

In [ ]:
# create a function to reset the multilevel columns after group by and aggregation
def reset_multi_level_columns(df):
    mean_columns = ['_'.join(filter(None, col)) for col in df.columns]
    df.columns=mean_columns
    return df

In [ ]:
# create a dataframe to look at the mean, median value group by indicator and response, reset multilevel columns
mean_data_value_by_indicator_response = results_df.groupby(['indicator','response']).agg({'data_value': ['mean', 'median', 'count']}).reset_index()
mean_data_value_by_indicator_response  = reset_multi_level_columns(mean_data_value_by_indicator_response)
mean_data_value_by_indicator_response.head(2)

In [ ]:
# plot the median values of data_value group by indicator and response
fig = px.bar(mean_data_value_by_indicator_response,
   x='response', y='data_value_median', color='indicator', barmode='group')
fig.update_layout(title= 'Bar plot of median data_value group by indicator and response')
fig.show()

### Observation:
* For disability by age group showed the disability rate is higher as age increases;
* For diability by race/ethnicity group, the disability rate for American Indian/ Native Hawaiian/multirace are the top three high disability rate group, white non hispanic being the lowest rate; This might related to the limitation of good health care for these groups or affortability to medicare.
* For sex group, the disability rate of female vs male are close;
* For verteran status group, the disability rate of veteran is slightly higher than non-veteran; The slightly higher disability of veteran might due to injury from war.

## Investigate the median data_value group by indicator and stratification1

In [ ]:
mean_data_value_by_stratification1 = results_df.groupby(['indicator','stratification1']).agg({'data_value': ['mean', 'median', 'count']}).reset_index()
mean_data_value_by_stratification1 = reset_multi_level_columns(mean_data_value_by_stratification1)
mean_data_value_by_stratification1.head(2)

In [ ]:
# Bar plot of median data_value gropu by indicator and straitification1
fig = px.bar(mean_data_value_by_stratification1,
   x='stratification1', y='data_value_median', color = 'indicator', barmode='group')
fig.update_layout(title= 'Bar plot of median_data_value')
fig.show()

### Observation:
* The median disability data_value is similar among four indicator categories (age group, race/ethnicity, sex, veteran_status)
    * hearing disability rate is higher for veteran status group, this might be associated with veteran group that suffer from hearing injury from war;
    * mobility disability is higher in age group, rate/ethnicity and veteran status group, which will look a little more in-depth;
* The no disability rate is the highest ~70-73%, and any disability rate is about 27-30%
* Within the disability categories, the cognitive disability and mobility disability are among the highest. 

In [ ]:
mean_data_value_by_stratification1_response = results_df.groupby(['response','stratification1']).agg({'data_value': ['mean', 'median', 'count']}).reset_index()
mean_data_value_by_stratification1_response = reset_multi_level_columns(mean_data_value_by_stratification1_response)
mean_data_value_by_stratification1_response.head(2)

In [ ]:
# Bar plot of median data_value gropu by indicator and straitification1
fig = px.bar(mean_data_value_by_stratification1_response,
   x='stratification1', y='data_value_median', color = 'response', barmode='group')
fig.update_layout(title= 'Bar plot of median_data_value by response and stratification1')
fig.show()

### Observations(by toddling the legends):
* Look the three age groups, the higher disability rate in hearing disbility, mobility, vision disability and independent living disability increases in elderly group (65+); the cognititive disability, however, decreases as age increases.
* Look at the race/ethnicity groups, the lowest disability rate is Asian in all sub categories; White nonhistpanic group has the 2nd lowest disability. Nativethe rest of the groups have similar rate; Other/multirace being the highest rate for disability;
* For sex groups, the disability rate rate are similar; hearing disability for male is higher than female, indepedent living disability for female is higher than male.
* For veteran status groups, the hearing and mobidity disability of veteran is higher than non-veratan, the rest disability categories are similar

## Check the number of entries group by indicator and stratification1 to see if there's any significant imbalance of data

In [ ]:
# Bar plot of median data_value gropu by indicator and stratification1
fig = px.bar(mean_data_value_by_stratification1_response,
   x='stratification1', y='data_value_count', color = 'response', barmode='group')
fig.update_layout(title= 'Bar plot of counts by response and stratification1')
fig.show()

### Observations on imbalanced data:
* For race/ethnicity groups, the population of Asian and Native Hawaiian are significantly less as compared to other groups; 
* The rest of the groups did not observe a significant less population/counts

## Disability median data_value by state

In [ ]:
data_value_by_state = results_df.groupby('locationabbr').agg({'data_value':'median'}).reset_index().sort_values(by='data_value', ascending=False)

fig = px.bar(data_value_by_state.iloc[:15,:],
   x='locationabbr', y='data_value', )
fig.update_layout(title= 'Bar plot of top 15 disability median_data_value by state')
fig.show()

In [ ]:
fig = px.bar(data_value_by_state.iloc[-15:,:],
   x='locationabbr', y='data_value', )
fig.update_layout(title= 'Bar plot of top 15 Lowest disability median_data_value by state')
fig.update_yaxes(range=[0, 45])

fig.show()

# Check fairness and mitigate bias
* check and mitigate the bias in Race/Ethnicity, especially for Asian and Native Hawaiian groups


In [ ]:
#!pip install aif360

In [ ]:
import pandas as pd
from aif360.datasets import StandardDataset
from aif360.metrics import BinaryLabelDatasetMetric
from aif360.algorithms.preprocessing import Reweighing


### Convert catogorical values to dummified columns

In [ ]:
categorical_cols = ['stratificationid1','responseid','indicatorid']
results_df_encoded = pd.get_dummies(results_df,
                                    columns = categorical_cols,
                                    drop_first=False)
results_df_encoded.head(2)

In [ ]:
# convert the dummy cols from False/True to numeric 0/1
dummy_cols = [i for i in results_df_encoded.columns if i not in results_df.columns]
results_df_encoded  = results_df_encoded[dummy_cols].map(lambda x: {False:0, True:1}.get(x, x))

In [ ]:
# drop the binary encoded column such as sex, age, veteran and disability vs nodisability
results_df_encoded = results_df_encoded.drop(['responseid_SEX01','responseid_AGE02', 'responseid_VET1','stratificationid1_NODIS'], axis=1)

In [ ]:
# combine selected features with dummy features
results_df_dummy_clean = pd.concat([results_df[['locationid', 'data_value']], results_df_encoded], axis=1)
results_df_dummy_clean = results_df_dummy_clean.dropna(subset=['data_value'])
print('Data size after drop null value in data_value',results_df_dummy_clean.shape[0])
results_df_dummy_clean.head(2)

### Save the cleaned data as csv files 

In [ ]:
results_df_dummy_clean.to_csv('cdc_2021_preventative_disability_status_type_demographic_data_one_hot_encoded.csv')

## Check correlation heatmap

In [ ]:
corr_matrix = results_df_dummy_clean.corr()

plt.figure(figsize = (15,10))
sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', fmt = '.2f')
plt.title('Feature correlation heatmap')
plt.show()

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import StandardScaler
import numpy as np
#set random seed
np.random.seed(40)

## Data train and test split

In [ ]:
# Define features and target variables 
target_cols = [col for col in results_df_dummy_clean.columns if 'stratificationid1_' in col]
X = results_df_dummy_clean.drop(target_cols, axis=1)
y = results_df_dummy_clean[target_cols]

# standard scaler
scaler = StandardScaler()
X_scale = scaler.fit_transform(X)

# train and test split
X_train, X_test, y_train, y_test = train_test_split(X_scale, y, test_size=0.2, random_state=42)

print(f'Train dataset size : {X_train.shape} and Test dataset size: {X_test.shape}')

## Model selection and evaluation

In [ ]:
from sklearn.ensemble import RandomForestClassifier

# Initialize and train the model
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Make predictions
y_pred = model.predict(X_test)

# Compute accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.2f}")

# Detailed classification report
print(classification_report(y_test, y_pred, target_names=y_train.columns))


### Baseline model can achieve 0.91 F1 score for correctively predict disability, but low f1 score for predict a specific diability type

## Grid search for best estimator

In [ ]:
# Define the parameter grid
param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'bootstrap': [True, False]
}

# Initialize the Random Forest Classifier
rf = RandomForestClassifier(random_state=42)

# Perform Grid Search
grid_search = GridSearchCV(rf, param_grid, cv=5, scoring='accuracy', n_jobs=-1)
grid_search.fit(X_train, y_train)

# Get the best parameters
print("Best hyperparameters:", grid_search.best_params_)


In [ ]:
best_model = grid_search.best_estimator_
y_pred = best_model.predict(X_test)

# Evaluate performance
# Compute accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.2f}")

# Detailed classification report
print(classification_report(y_test, y_pred, target_names=y_train.columns))


In [ ]:
results_df_dummy_clean_droped = results_df_dummy_clean.drop(['stratificationid1_DISABL'], axis=1)

## Prepare train and test dataset

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import numpy as np
#set random seed for python's build-in random module
np.random.seed(42)

In [ ]:
results_df.stratificationid1.unique()

In [ ]:
# Define features and target variable
drop_cols = [col for col in results_df_dummy_clean.columns if 'stratificationid1_' in col or 'data_value' in col]
target_cols = ['stratificationid1_HEARDIS','stratificationid1_MOBDIS','straitificationid1_COGDIS']
X = results_df_dummy_clean.drop(drop_cols, axis=1)
y = results_df_dummy_clean[target_cols]

# Split into 80% training and 20% test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Check the shape of the datasets
print(f"Training set shape: {X_train.shape}, Testing set shape: {X_test.shape}")


In [ ]:
# Since most of the features are dummified features but the locationID is not, we'll normalize the data using standardscaler

scaler = StandardScaler()
X_train_scaled, X_test_scaled = scaler.fit_transform(X_train), scaler.fit_transform(X_test)


## Model selection and training

For regression model, we can experiment LinearRegression, RandomForestRegressor from Sklearn; we'll compare these two and select a better model based on metrics like MSE and R2 score

In [ ]:
# Random forestest regressor
from sklearn.ensemble import RandomForestClassifier

# Grid search for hyper parameters optimization
from sklearn.model_selection import GridSearchCV

# Model evaluation metrics
from sklearn.metrics import mean_squared_error, r2_score

In [ ]:
# log the metrics 
metric_logs = pd.DataFrame()
metric_logs.loc[0, 'model'] = 'linear_regression'
metric_logs.loc[0, 'mse_test'] = round(mse,2)
metric_logs.loc[0, 'r2_test'] = round(r2,2)
metric_logs

In [ ]:
# initialize and train the model
rf = RandomForestClassifier(n_estimators = 100, random_state=42)
rf.fit(X_train_scaled, y_train)

# prediction
y_pred = rf.predict(X_test_scaled)

In [ ]:
# calculate MSE and R2 score
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print('Random Forest Regressor metrics')
print(f"Mean Squared Error: {mse:.2f}")
print(f"R² Score: {r2:.2f}")

In [ ]:
# log the metric and add to the log dataframe
i = len(metric_logs)
metric_logs.loc[i, 'model'] = 'random_forest_bs'
metric_logs.loc[i, 'mse_test'] = round(mse,2)
metric_logs.loc[i, 'r2_test'] = round(r2,2)
metric_logs

In [ ]:
param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5, 10]
}

grid_search = GridSearchCV(RandomForestRegressor(random_state=42), param_grid, cv=5, scoring='r2')
grid_search.fit(X_train_scaled, y_train)

# Best parameters
print("Best parameters:", grid_search.best_params_)


In [ ]:
# make prediction
y_pred_grid = grid_search.predict(X_test_scaled)

# calculate MSE and R2 score
mse = mean_squared_error(y_test, y_pred_grid)
r2 = r2_score(y_test, y_pred_grid)

print('Random Forest Regressor grid_search best estimator metrics')
print(f"Mean Squared Error: {mse:.2f}")
print(f"R² Score: {r2:.2f}")



In [ ]:
# log the metric and add to the log dataframe
i = len(metric_logs)
metric_logs.loc[i, 'model'] = 'random_forest_gv'
metric_logs.loc[i, 'mse_test'] = round(mse,2)
metric_logs.loc[i, 'r2_test'] = round(r2,2)
metric_logs

# Gridsearch best estimator improved slightly the f1 score in predicting specific type of disability and remain the same f1-score for corectively predicting disability

### Visualize feature importance

In [ ]:
# extract feature importance
best_model = grid_search.best_estimator_
feature_importance = best_model.feature_importances_

#create dataframe
importance_df = pd.DataFrame({'Feature': X.columns, 'Importance': feature_importance})
importance_df = importance_df.sort_values(by='Importance', ascending=False)

# Plot feature importance
plt.figure(figsize=(10, 6))
plt.barh(importance_df['Feature'], importance_df['Importance'])
plt.xlabel("Feature Importance Score")
plt.ylabel("Features")
plt.title("Feature Importance in Random Forest")
plt.gca().invert_yaxis()  # Highest importance on top
plt.show()


# The most influence feature is `data_value` followed by `locatoinid`, the demographic features are not as important in predicting disability status and types

# Conclusion 

From this project, we have explore the hidden trends between disability status and types across different demographic groups, we had observed the following important trends that can provide insights for developing better interventions and service. Through the data pipeline, we explore data type, drop nul value in 'data_value', convert categorical features to encoded features, drop highly correlated features to decrease overfit risks, and visualize the trends to provide better insights:
* **Age**:
    * Older adults have significantly higher disability rates compared to younger age groups. 
* **Race/Ethnicity**:
    * Some racial groups, including American Indian or Alaska Native and Other/Multirace, have higher disability prevalence. 
* **Socioeconomic Factors**:
    * Individuals with lower education levels and from minority racial backgrounds may experience higher disability severity. 
* **Sex**:
    * Women may be more likely to experience certain types of disabilities compared to men. 
* **Disability Types**:
    * Common types include difficulties with walking or mobility, independent living (errands alone), and cognitive difficulties.
* We have siginificantly less data for Asian and Native Hawaiian groups, which is likely due to being minority in US.

We also prepared train and test dataset, and defined the disability status and disability types as our target variables, and use  `data_value`, `locationid` as well as `one_hot_encoded` `response` and `indicator` classes as features. We developed a multiclassification model using `RandomForestClassifier` and optimize the hyper parameters through `grid_search` and evaluate the model metrics for disability status and type via `accuracy` and `f1_score`. We concluded that the best model is able to achieve 91% F1_score in predict correctly the disability status, but the model has poor performance in predicting disability types correctly. 

## Future plan
* Given more time, I will attempt to explore:
    * Feature engineering such as `2d order polynomial features`
    * Different model type such as `XGBoost` or `SVM` algorithms
    * Explore if there's any additional data for disbility prevalance that can add to this study
  
